<a href="https://colab.research.google.com/github/pamelaAHeredia/prediccion_desperdicio_de_comida/blob/main/merged_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import pandas as pd
import plotly.express as px

# Datasets

https://www.fao.org/faostat/en/#data/

In [246]:
url = "https://raw.githubusercontent.com/pamelaAHeredia/prediccion_desperdicio_de_comida/main/Production_Indices.csv"

production_indices = pd.read_csv(url, delimiter=',')

In [247]:
url = "https://raw.githubusercontent.com/pamelaAHeredia/prediccion_desperdicio_de_comida/main/Data.csv"

data = pd.read_csv(url, delimiter=',')

In [248]:
url = "https://raw.githubusercontent.com/pamelaAHeredia/prediccion_desperdicio_de_comida/main/Cost_and_Affordability_of_a_Healthy%20Diet.csv"

cost_diet = pd.read_csv(url, delimiter=',')

In [249]:
url = "https://raw.githubusercontent.com/pamelaAHeredia/prediccion_desperdicio_de_comida/main/Annual_Pupulation.csv"

annual_population = pd.read_csv(url, delimiter=',')

In [250]:
data_df = data.copy()
prod_df = production_indices.copy()
cost_diet_df= cost_diet.copy()
annual_pop_df = annual_population.copy()

# Funciones útiles

In [251]:
# Todos los dataset son de la misma fuente, tienen los mismos formatos de nombre de columnas

def normalize_columns(dataset, ds_name):
  # Reemplazar todas las mayúsculas en los nombres de las columnas por minúsculas yt los espacios por guines bajos en production_df
  dataset.columns = dataset.columns.str.lower()
  dataset.columns = dataset.columns.str.replace(' ', '_')

  #renombrar columnas para poder realizar el merge
  dataset = dataset.rename(columns={'area_code_(m49)': 'm49_code'})
  dataset = dataset.rename(columns={'year_code': 'year'})
  dataset = dataset.rename(columns={'element_code': ds_name+'_element_code'})
  dataset = dataset.rename(columns={'item_code': ds_name+'_item_code'})
  dataset = dataset.rename(columns={'value': ds_name+'_value'})
  dataset = dataset.rename(columns={'element': ds_name+'_element'})
  dataset = dataset.rename(columns={'item': ds_name+'_item'})
  dataset = dataset.rename(columns={'domain': ds_name+'_domain'})
  dataset = dataset.rename(columns={'domain_code': ds_name+'_domain_code'})

  return dataset

def m49code_to_dict(dataset, column):
  # Agrupar por 'm49_code' y obtener los países únicos para cada código
  grouped = dataset.groupby('m49_code')[column].apply(lambda x: list(x.unique()))

  # Convertir el resultado a un diccionario
  m49_country_dict = grouped.to_dict()

  # Imprimir el diccionario resultante
  return m49_country_dict

# Limpieza del dataset Data

In [215]:
# Info de las columnas

data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25416 entries, 0 to 25415
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   m49_code                  25416 non-null  int64  
 1   country                   25416 non-null  object 
 2   region                    1214 non-null   object 
 3   cpc_code                  25416 non-null  object 
 4   commodity                 25416 non-null  object 
 5   year                      25416 non-null  int64  
 6   loss_percentage           25416 non-null  float64
 7   loss_percentage_original  25416 non-null  object 
 8   loss_quantity             539 non-null    object 
 9   activity                  22608 non-null  object 
 10  food_supply_stage         22025 non-null  object 
 11  treatment                 1320 non-null   object 
 12  cause_of_loss             1002 non-null   object 
 13  sample_size               1192 non-null   object 
 14  method

In [216]:
#Columnas a eliminar en el archivo data: 'cpc_code', 'method_data_collection', 'url', 'notes'
columnas_a_eliminar = ['method_data_collection', 'url','notes','region',
                       'cause_of_loss','sample_size','treatment','reference', 'food_supply_stage','loss_quantity','activity']
data_df = data.drop(columns=columnas_a_eliminar)

In [217]:
data_df.isnull().sum()

m49_code                    0
country                     0
cpc_code                    0
commodity                   0
year                        0
loss_percentage             0
loss_percentage_original    0
dtype: int64

In [218]:
data_df.head()

,m49_code,country,cpc_code,commodity,year,loss_percentage,loss_percentage_original
0,104,Myanmar,23161.02,"Rice, milled",2015,1.78,1.78%
1,104,Myanmar,23161.02,"Rice, milled",2015,11.77,11.77%
2,104,Myanmar,23161.02,"Rice, milled",2015,5.88,5.88%
3,104,Myanmar,23161.02,"Rice, milled",2015,3.57,3.57%
4,104,Myanmar,23161.02,"Rice, milled",2015,17.65,17.65%


In [219]:
data_numerical_columns = data_df.select_dtypes(include=['number'])

print(data_numerical_columns)

       m49_code  year  loss_percentage
0           104  2015          1.78000
1           104  2015         11.77000
2           104  2015          5.88000
3           104  2015          3.57000
4           104  2015         17.65000
...         ...   ...              ...
25411       894  2000          2.50000
25412       894  2000          2.50000
25413       894  2000          2.38075
25414       894  2000          3.44008
25415       894  2000          1.27433

[25416 rows x 3 columns]


In [220]:
# generar diccionario con los códigos m49 y sus respectivos países

data_m49code_dict = m49code_to_dict(data_df, 'country')

print(data_m49code_dict)

{2: ['Africa'], 11: ['Western Africa'], 12: ['Algeria'], 15: ['Northern Africa'], 21: ['Northern America'], 24: ['Angola'], 31: ['Azerbaijan'], 32: ['Argentina'], 34: ['Southern Asia'], 35: ['South-Eastern Asia'], 36: ['Australia'], 48: ['Bahrain'], 50: ['Bangladesh'], 51: ['Armenia'], 53: ['Australia and New Zealand'], 68: ['Bolivia (Plurinational State of)'], 72: ['Botswana'], 76: ['Brazil'], 104: ['Myanmar'], 108: ['Burundi'], 116: ['Cambodia'], 120: ['Cameroon'], 124: ['Canada'], 143: ['Central Asia'], 144: ['Sri Lanka'], 145: ['Western Asia'], 148: ['Chad'], 150: ['Europe'], 152: ['Chile'], 156: ['China'], 170: ['Colombia'], 180: ['Democratic Republic of the Congo'], 188: ['Costa Rica'], 192: ['Cuba'], 202: ['Sub-Saharan Africa'], 204: ['Benin'], 208: ['Denmark'], 218: ['Ecuador'], 222: ['El Salvador'], 231: ['Ethiopia'], 232: ['Eritrea'], 242: ['Fiji'], 246: ['Finland'], 250: ['France'], 266: ['Gabon'], 270: ['Gambia'], 275: ['Palestine'], 276: ['Germany'], 288: ['Ghana'], 320: [

# Limpieza dataset Production_indices

In [252]:
prod_df.columns

Index(['Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code',
       'Element', 'Item Code (CPC)', 'Item', 'Year Code', 'Year', 'Unit',
       'Value', 'Flag', 'Flag Description'],
      dtype='object')

In [253]:
# Columnas a eliminar en el archivo Production_indices

columnas_a_eliminar = ['Unit', 'Year', 'Flag', 'Flag Description']
prod_df = prod_df.drop(columns=columnas_a_eliminar)

In [254]:
#normalizar formato de columnas

prod_df = normalize_columns(prod_df, 'prod')

prod_df.columns

Index(['prod_domain_code', 'prod_domain', 'm49_code', 'area',
       'prod_element_code', 'prod_element', 'item_code_(cpc)', 'prod_item',
       'year', 'prod_value'],
      dtype='object')

In [255]:
prod_df.isnull().sum()

prod_domain_code     0
prod_domain          0
m49_code             0
area                 0
prod_element_code    0
prod_element         0
item_code_(cpc)      0
prod_item            0
year                 0
prod_value           0
dtype: int64

In [256]:
prod_df.head()

,prod_domain_code,prod_domain,m49_code,area,prod_element_code,prod_element,item_code_(cpc),prod_item,year,prod_value
0,QI,Production Indices,4,Afghanistan,432,Gross Production Index Number (2014-2016 = 100),F2051,Agriculture,2000,68.63
1,QI,Production Indices,4,Afghanistan,432,Gross Production Index Number (2014-2016 = 100),F2051,Agriculture,2001,62.39
2,QI,Production Indices,4,Afghanistan,432,Gross Production Index Number (2014-2016 = 100),F2051,Agriculture,2002,74.96
3,QI,Production Indices,4,Afghanistan,432,Gross Production Index Number (2014-2016 = 100),F2051,Agriculture,2003,76.62
4,QI,Production Indices,4,Afghanistan,432,Gross Production Index Number (2014-2016 = 100),F2051,Agriculture,2004,74.66


In [257]:
prod_numerical_columns = prod_df.select_dtypes(include=['number'])

print(prod_numerical_columns)

       m49_code  prod_element_code  year  prod_value
0             4                432  2000       68.63
1             4                432  2001       62.39
2             4                432  2002       74.96
3             4                432  2003       76.62
4             4                432  2004       74.66
...         ...                ...   ...         ...
93587       716                434  2018       94.49
93588       716                434  2019       87.77
93589       716                434  2020       92.43
93590       716                434  2021       91.56
93591       716                434  2022       90.57

[93592 rows x 4 columns]


In [258]:
# generar diccionario con los códigos m49 y sus respectivos países

prod_m49code_dict = m49code_to_dict(prod_df, 'area')

print(prod_m49code_dict)

{4: ['Afghanistan'], 8: ['Albania'], 12: ['Algeria'], 24: ['Angola'], 28: ['Antigua and Barbuda'], 31: ['Azerbaijan'], 32: ['Argentina'], 36: ['Australia'], 40: ['Austria'], 44: ['Bahamas'], 48: ['Bahrain'], 50: ['Bangladesh'], 51: ['Armenia'], 52: ['Barbados'], 56: ['Belgium'], 64: ['Bhutan'], 68: ['Bolivia (Plurinational State of)'], 70: ['Bosnia and Herzegovina'], 72: ['Botswana'], 76: ['Brazil'], 84: ['Belize'], 90: ['Solomon Islands'], 96: ['Brunei Darussalam'], 100: ['Bulgaria'], 104: ['Myanmar'], 108: ['Burundi'], 112: ['Belarus'], 116: ['Cambodia'], 120: ['Cameroon'], 124: ['Canada'], 132: ['Cabo Verde'], 140: ['Central African Republic'], 144: ['Sri Lanka'], 148: ['Chad'], 152: ['Chile'], 156: ['China, mainland'], 158: ['China, Taiwan Province of'], 159: ['China'], 170: ['Colombia'], 174: ['Comoros'], 178: ['Congo'], 180: ['Democratic Republic of the Congo'], 184: ['Cook Islands'], 188: ['Costa Rica'], 191: ['Croatia'], 192: ['Cuba'], 196: ['Cyprus'], 203: ['Czechia'], 204: ['

# Dataset cost_diet

In [259]:
cost_diet_df.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),8,Albania,6226,Value,7004,Cost of a healthy diet (PPP dollar per person ...,2017,2017,PPP dollar per person per day,3.952,E,Estimated value
1,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),8,Albania,6226,Value,7004,Cost of a healthy diet (PPP dollar per person ...,2018,2018,PPP dollar per person per day,4.069,E,Estimated value
2,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),8,Albania,6226,Value,7004,Cost of a healthy diet (PPP dollar per person ...,2019,2019,PPP dollar per person per day,4.262,E,Estimated value
3,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),8,Albania,6226,Value,7004,Cost of a healthy diet (PPP dollar per person ...,2020,2020,PPP dollar per person per day,4.280,E,Estimated value
4,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),8,Albania,6226,Value,7004,Cost of a healthy diet (PPP dollar per person ...,2021,2021,PPP dollar per person per day,4.388,E,Estimated value


In [260]:
cost_diet_df.isnull().sum()

Domain Code           0
Domain                0
Area Code (M49)       0
Area                  0
Element Code          0
Element               0
Item Code             0
Item                  0
Year Code             0
Year                  0
Unit                  0
Value               276
Flag                  0
Flag Description      0
dtype: int64

In [261]:
# Eliminar filas con valores faltantes y columnas
columnas_a_eliminar = ['Year','Flag','Flag Description']
cost_df = cost_diet_df.drop(columns=columnas_a_eliminar)

cost_df = cost_df.dropna(subset=['Value'])

cost_df.columns

Index(['Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code',
       'Element', 'Item Code', 'Item', 'Year Code', 'Unit', 'Value'],
      dtype='object')

In [262]:
# Normalizar formato de columnas para poder hacer el merge

cost_df = normalize_columns(cost_df,'cost')

cost_df.columns

Index(['cost_domain_code', 'cost_domain', 'm49_code', 'area',
       'cost_element_code', 'cost_element', 'cost_item_code', 'cost_item',
       'year', 'unit', 'cost_value'],
      dtype='object')

In [263]:
# extraer columnas con valores numéricas para visualizar correlaciones entre los datos del dataset resultante del merge

cost_numerical_columns = cost_df.select_dtypes(include=['number'])

print(cost_numerical_columns)

      m49_code  cost_element_code  cost_item_code  year  cost_value
0            8               6226            7004  2017       3.952
1            8               6226            7004  2018       4.069
2            8               6226            7004  2019       4.262
3            8               6226            7004  2020       4.280
4            8               6226            7004  2021       4.388
...        ...                ...             ...   ...         ...
3506       716               6226            7010  2017       0.968
3507       716               6226            7011  2017       0.513
3508       716               6226            7012  2017       0.144
3509       716               6121            7005  2017      67.800
3514       716               6132            7006  2017      10.000

[3243 rows x 5 columns]


In [264]:
# generar diccionario con los códigos m49 y sus respectivos países

cost_m49code_dict = m49code_to_dict(cost_df, 'area')

print(cost_m49code_dict)

{8: ['Albania'], 12: ['Algeria'], 24: ['Angola'], 28: ['Antigua and Barbuda'], 31: ['Azerbaijan'], 32: ['Argentina'], 36: ['Australia'], 40: ['Austria'], 44: ['Bahamas'], 48: ['Bahrain'], 50: ['Bangladesh'], 51: ['Armenia'], 56: ['Belgium'], 60: ['Bermuda'], 64: ['Bhutan'], 68: ['Bolivia (Plurinational State of)'], 70: ['Bosnia and Herzegovina'], 72: ['Botswana'], 76: ['Brazil'], 84: ['Belize'], 92: ['British Virgin Islands'], 96: ['Brunei Darussalam'], 100: ['Bulgaria'], 104: ['Myanmar'], 108: ['Burundi'], 112: ['Belarus'], 116: ['Cambodia'], 120: ['Cameroon'], 124: ['Canada'], 132: ['Cabo Verde'], 136: ['Cayman Islands'], 140: ['Central African Republic'], 144: ['Sri Lanka'], 148: ['Chad'], 152: ['Chile'], 156: ['China, mainland'], 158: ['China, Taiwan Province of'], 170: ['Colombia'], 178: ['Congo'], 180: ['Democratic Republic of the Congo'], 188: ['Costa Rica'], 191: ['Croatia'], 196: ['Cyprus'], 203: ['Czechia'], 204: ['Benin'], 208: ['Denmark'], 212: ['Dominica'], 214: ['Dominica

# Annual Population

In [265]:
annual_pop_df.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Value
0,OA,Annual population,4,Afghanistan,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2001,2001,19688.632
1,OA,Annual population,4,Afghanistan,512,Total Population - Male,3010,Population - Est. & Proj.,2001,2001,9895.467
2,OA,Annual population,4,Afghanistan,513,Total Population - Female,3010,Population - Est. & Proj.,2001,2001,9793.166
3,OA,Annual population,4,Afghanistan,551,Rural population,3010,Population - Est. & Proj.,2001,2001,16318.324
4,OA,Annual population,4,Afghanistan,561,Urban population,3010,Population - Est. & Proj.,2001,2001,4648.139


In [266]:
annual_pop_df.isnull().sum()

Domain Code        0
Domain             0
Area Code (M49)    0
Area               0
Element Code       0
Element            0
Item Code          0
Item               0
Year Code          0
Year               0
Value              0
dtype: int64

In [267]:
# Eliminar columnas repetidas
columnas_a_eliminar = ['Year']
pop_df = annual_pop_df.drop(columns=columnas_a_eliminar)

In [268]:
# Normalizar formato de columnas

pop_df = normalize_columns(pop_df, 'pop')

pop_df.columns

Index(['pop_domain_code', 'pop_domain', 'm49_code', 'area', 'pop_element_code',
       'pop_element', 'pop_item_code', 'pop_item', 'year', 'pop_value'],
      dtype='object')

In [269]:
# extraer columnas con valores numéricas para visualizar correlaciones entre los datos del dataset resultante del merge

pop_numerical_columns = pop_df.select_dtypes(include=['number'])

print(pop_numerical_columns)

       m49_code  pop_element_code  pop_item_code  year  pop_value
0             4               511           3010  2001  19688.632
1             4               512           3010  2001   9895.467
2             4               513           3010  2001   9793.166
3             4               551           3010  2001  16318.324
4             4               561           3010  2001   4648.139
...         ...               ...            ...   ...        ...
24388       716               511           3010  2021  15993.524
24389       716               512           3010  2021   7543.690
24390       716               513           3010  2021   8449.834
24391       716               551           3010  2021  12226.340
24392       716               561           3010  2021   5834.113

[24393 rows x 5 columns]


In [270]:
# generar diccionario con los códigos m49 y sus respectivos países

pop_m49code_dict = m49code_to_dict(pop_df, 'area')

print(pop_m49code_dict)

{4: ['Afghanistan'], 8: ['Albania'], 12: ['Algeria'], 16: ['American Samoa'], 20: ['Andorra'], 24: ['Angola'], 28: ['Antigua and Barbuda'], 31: ['Azerbaijan'], 32: ['Argentina'], 36: ['Australia'], 40: ['Austria'], 44: ['Bahamas'], 48: ['Bahrain'], 50: ['Bangladesh'], 51: ['Armenia'], 52: ['Barbados'], 56: ['Belgium'], 60: ['Bermuda'], 64: ['Bhutan'], 68: ['Bolivia (Plurinational State of)'], 70: ['Bosnia and Herzegovina'], 72: ['Botswana'], 76: ['Brazil'], 84: ['Belize'], 90: ['Solomon Islands'], 92: ['British Virgin Islands'], 96: ['Brunei Darussalam'], 100: ['Bulgaria'], 104: ['Myanmar'], 108: ['Burundi'], 112: ['Belarus'], 116: ['Cambodia'], 120: ['Cameroon'], 124: ['Canada'], 132: ['Cabo Verde'], 136: ['Cayman Islands'], 140: ['Central African Republic'], 144: ['Sri Lanka'], 148: ['Chad'], 152: ['Chile'], 156: ['China, mainland'], 158: ['China, Taiwan Province of'], 159: ['China'], 170: ['Colombia'], 174: ['Comoros'], 175: ['Mayotte'], 178: ['Congo'], 180: ['Democratic Republic of

# Merge entre Datasets

In [271]:
# Merge entre DS

merged_df = pd.merge(data_df, prod_df, on=['m49_code', 'year'])
merged_df = pd.merge(merged_df, pop_df, on=['m49_code', 'year'])
merged_df = pd.merge(merged_df, cost_df, on=['m49_code', 'year'])

merged_df.columns

Index(['m49_code', 'country', 'region', 'cpc_code', 'commodity', 'year',
       'loss_percentage', 'loss_percentage_original', 'loss_quantity',
       'activity', 'food_supply_stage', 'treatment', 'cause_of_loss',
       'sample_size', 'method_data_collection', 'reference', 'url', 'notes',
       'prod_domain_code', 'prod_domain', 'area_x', 'prod_element_code',
       'prod_element', 'item_code_(cpc)', 'prod_item', 'prod_value',
       'pop_domain_code', 'pop_domain', 'area_y', 'pop_element_code',
       'pop_element', 'pop_item_code', 'pop_item', 'pop_value',
       'cost_domain_code', 'cost_domain', 'area', 'cost_element_code',
       'cost_element', 'cost_item_code', 'cost_item', 'unit', 'cost_value'],
      dtype='object')

In [272]:
# matriz de correlación

numerical_columns_df = merged_df.select_dtypes(include=['number'])

# Calcular la matriz de correlación entre las variables numéricas en el DataFrame fusionado
correlation_matrix = numerical_columns_df.corr()

# Visualizar la matriz de correlación
print(correlation_matrix)


                       m49_code          year  loss_percentage  \
m49_code           1.000000e+00 -5.906621e-02     1.256801e-01   
year              -5.906621e-02  1.000000e+00    -1.175768e-01   
loss_percentage    1.256801e-01 -1.175768e-01     1.000000e+00   
prod_element_code -4.008947e-12  2.478176e-13    -3.717637e-13   
prod_value         3.985397e-02  1.465197e-01    -5.511159e-02   
pop_element_code  -6.952790e-13  9.954678e-14    -2.335863e-13   
pop_item_code               NaN           NaN              NaN   
pop_value          1.050239e-01  3.032031e-02     1.821541e-01   
cost_element_code  1.600026e-02 -3.601246e-01     3.176387e-02   
cost_item_code     2.181176e-02 -5.130571e-01     4.186103e-02   
cost_value        -6.465731e-02  3.094020e-01    -8.718064e-02   

                   prod_element_code    prod_value  pop_element_code  \
m49_code               -4.008947e-12  3.985397e-02     -6.952790e-13   
year                    2.478176e-13  1.465197e-01      9.95467

In [30]:
merged_df['pop_item_code'].isnull().sum()

0

In [47]:
# Merge de diccionarios con códigos m49, sin claves repetidas. Para completar el dataset faltante, que no tiene

merged_m49codes = {}

# Fusionar los diccionarios sin claves repetidas
for d in [pop_m49code_dict, prod_m49code_dict, cost_m49code_dict, data_m49code_dict]:
    for key, value in d.items():
        if key not in merged_m49codes:
            merged_m49codes[key] = value

print(merged_m49codes)

{4: ['Afghanistan'], 8: ['Albania'], 12: ['Algeria'], 16: ['American Samoa'], 20: ['Andorra'], 24: ['Angola'], 28: ['Antigua and Barbuda'], 31: ['Azerbaijan'], 32: ['Argentina'], 36: ['Australia'], 40: ['Austria'], 44: ['Bahamas'], 48: ['Bahrain'], 50: ['Bangladesh'], 51: ['Armenia'], 52: ['Barbados'], 56: ['Belgium'], 60: ['Bermuda'], 64: ['Bhutan'], 68: ['Bolivia (Plurinational State of)'], 70: ['Bosnia and Herzegovina'], 72: ['Botswana'], 76: ['Brazil'], 84: ['Belize'], 90: ['Solomon Islands'], 92: ['British Virgin Islands'], 96: ['Brunei Darussalam'], 100: ['Bulgaria'], 104: ['Myanmar'], 108: ['Burundi'], 112: ['Belarus'], 116: ['Cambodia'], 120: ['Cameroon'], 124: ['Canada'], 132: ['Cabo Verde'], 136: ['Cayman Islands'], 140: ['Central African Republic'], 144: ['Sri Lanka'], 148: ['Chad'], 152: ['Chile'], 156: ['China, mainland'], 158: ['China, Taiwan Province of'], 159: ['China'], 170: ['Colombia'], 174: ['Comoros'], 175: ['Mayotte'], 178: ['Congo'], 180: ['Democratic Republic of

In [244]:
# Códigos m49

unique_codes = merged_df['m49_code'].unique()

print(unique_codes)

[108 120 144 148 156 180 204 208 231  24 246 270 275 288  31 324  36 380
 384 400 404 426 430 450 454 466 478  50 504 508 516 524 562 566 578 586
 608 624 646 682 686 694 710 716  72 729 748 752 788 800 818 826 834 840
 854 894]


In [275]:
# Agrupar países por regiones de acuerdo al código m49

# Me faltan datos de países en uno de los datasets, por lo que el merge elimina muchas filas y el dataset resultante queda a medias.

m49_to_area_subarea = {
    108: ("África Oriental", "África"),
    120: ("África Central", "África"),
    144: ("Asia Meridional", "Asia"),
    148: ("África Central", "África"),
    156: ("Asia Oriental", "Asia"),
    180: ("África Central", "África"),
    204: ("África Occidental", "África"),
    208: ("Europa Septentrional", "Europa"),
    231: ("África Oriental", "África"),
    24: ("África Central", "África"),
    246: ("Europa Septentrional", "Europa"),
    270: ("África Occidental", "África"),
    275: ("Asia Occidental", "Asia"),
    288: ("África Occidental", "África"),
    31: ("Asia Occidental", "Asia"),
    324: ("África Occidental", "África"),
    36: ("Australia y Nueva Zelanda", "Oceanía"),
    380: ("Europa Meridional", "Europa"),
    384: ("África Occidental", "África"),
    400: ("Asia Occidental", "Asia"),
    404: ("África Oriental", "África"),
    426: ("África Meridional", "África"),
    430: ("África Occidental", "África"),
    450: ("África Oriental", "África"),
    454: ("África Oriental", "África"),
    466: ("África Occidental", "África"),
    478: ("África Occidental", "África"),
    50: ("Asia Meridional", "Asia"),
    504: ("África Septentrional", "África"),
    508: ("África Oriental", "África"),
    516: ("África Meridional", "África"),
    524: ("Asia Meridional", "Asia"),
    562: ("África Occidental", "África"),
    566: ("África Occidental", "África"),
    578: ("Europa Septentrional", "Europa"),
    586: ("Asia Meridional", "Asia"),
    608: ("Asia Sudoriental", "Asia"),
    624: ("África Occidental", "África"),
    646: ("África Oriental", "África"),
    682: ("Asia Occidental", "Asia"),
    686: ("África Occidental", "África"),
    694: ("África Occidental", "África"),
    710: ("África Meridional", "África"),
    716: ("África Oriental", "África"),
    72: ("África Meridional", "África"),
    729: ("África Septentrional", "África"),
    748: ("África Meridional", "África"),
    752: ("Europa Septentrional", "Europa"),
    788: ("África Septentrional", "África"),
    800: ("África Oriental", "África"),
    818: ("África Septentrional", "África"),
    826: ("Europa Septentrional", "Europa"),
    834: ("África Oriental", "África"),
    840: ("América del Norte", "América"),
    854: ("África Occidental", "África"),
    894: ("África Oriental", "África")
}
merged_df['continental_area'] = ""

merged_df['continental_area'] = merged_df['m49_code'].map(lambda x: m49_to_area_subarea[x][0])

print(merged_df[['continental_area', 'm49_code']])

        continental_area  m49_code
0        África Oriental       108
1        África Oriental       108
2        África Oriental       108
3        África Oriental       108
4        África Oriental       108
...                  ...       ...
2171065  África Oriental       894
2171066  África Oriental       894
2171067  África Oriental       894
2171068  África Oriental       894
2171069  África Oriental       894

[2171070 rows x 2 columns]


In [276]:
# filtrar por continente

asia = merged_df[merged_df['continental_area'].str.contains('Asia')]

In [278]:
# Gráfico de dispersión - a ver si hay aaaaaalgo por acá

fig = px.scatter(asia, x='year', y='cost_value', color='continental_area',
                 hover_name='country', log_x=True, size_max=60,
                 title='valor de producción y porcentaje de pérdida')
fig.update_xaxes(title_text='Año')
fig.update_yaxes(title_text='Costo de la dieta')
fig.show()